In [1]:
library(ez)
library(car)
library(nlme)
require(multcomp)
library(phia)

Registered S3 methods overwritten by 'lme4':
  method                          from
  cooks.distance.influence.merMod car 
  influence.merMod                car 
  dfbeta.influence.merMod         car 
  dfbetas.influence.merMod        car 

Loading required package: carData

Loading required package: multcomp

Loading required package: mvtnorm

Loading required package: survival

Loading required package: TH.data

Loading required package: MASS


Attaching package: ‘TH.data’


The following object is masked from ‘package:MASS’:

    geyser




In [2]:
df = read.csv("./out/speeds.csv")

In [3]:
# We are interested in the grip area which are caused by the fingers on the back. 
# Thus, we exclude the thumb in our ANOVAs. 
df = df[df$Finger != "Thumb_Fn",]

In [4]:
summary(df)

      Cond           Finger         PId        Phone        Speed_2D    
 seated :256   Index_Fn :128   Min.   : 3.00   N6 :128   Min.   :0.368  
 walking:256   Little_Fn:128   1st Qu.: 6.75   OPO:128   1st Qu.:1.042  
               Middle_Fn:128   Median :11.50   S3 :128   Median :1.419  
               Ring_Fn  :128   Mean   :11.62   S4 :128   Mean   :1.673  
               Thumb_Fn :  0   3rd Qu.:16.25             3rd Qu.:1.981  
                               Max.   :21.00             Max.   :5.602  

In [5]:
df$Finger <- factor(df$Finger)
df$Cond <- factor(df$Cond)
df$Phone <- factor(df$Phone)
df$PId <- factor(df$PId)

# ANOVA on all Speeds

In [6]:
aov<-ezANOVA(data = df,
       dv = Speed_2D,
       within = c(Finger, Phone, Cond),
       wid = PId,
       type = 3,
       detailed = F,
       return_aov = F)

aov

$ANOVA
             Effect DFn DFd          F            p p<.05        ges
2            Finger   3  45 136.205401 1.367086e-22     * 0.38248888
3             Phone   3  45  46.250327 8.506747e-14     * 0.43580697
4              Cond   1  15 412.273795 2.534452e-12     * 0.62334249
5      Finger:Phone   9 135  19.558041 1.181894e-20     * 0.04935845
6       Finger:Cond   3  45 127.330478 5.314120e-22     * 0.10415080
7        Phone:Cond   3  45  19.477115 3.063633e-08     * 0.28333166
8 Finger:Phone:Cond   9 135   9.450177 4.514161e-11     * 0.02346037

$`Mauchly's Test for Sphericity`
             Effect            W            p p<.05
2            Finger 3.088756e-01 6.751039e-03     *
3             Phone 5.311440e-01 1.238795e-01      
5      Finger:Phone 3.241307e-05 1.084135e-08     *
6       Finger:Cond 1.686756e-01 1.948340e-04     *
7        Phone:Cond 4.480127e-01 5.196669e-02      
8 Finger:Phone:Cond 1.426636e-05 4.657538e-10     *

$`Sphericity Corrections`
             Effect       GGe        p[GG] p[GG]<.05       HFe        p[HF]
2            Finger 0.5921593 2.983803e-14         * 0.6660769 9.118792e-16
3             Phone 0.7608042 5.361907e-11         * 0.9047471 1.104729e-12
5      Finger:Phone 0.3250199 4.169457e-08         * 0.4122708 9.612063e-10
6       Finger:Cond 0.5231882 1.585079e-12         * 0.5736482 1.562808e-13
7        Phone:Cond 0.7283719 1.546550e-06         * 0.8573845 2.391914e-07
8 Finger:Phone:Cond 0.3009383 1.182662e-04         * 0.3736541 2.467740e-05
  p[HF]<.05
2         *
3         *
5         *
6         *
7         *
8         *

In [7]:
aov$`Sphericity Corrections`$GGe * c(3, 3, 9, 3, 3, 9)
aov$`Sphericity Corrections`$GGe * c(45, 45, 135, 45, 45, 135)

[1] 1.776478 2.282413 2.925179 1.569565 2.185116 2.708445

[1] 26.64717 34.23619 43.87769 23.54347 32.77674 40.62667

In [8]:
lme_velocity <- lme(Speed_2D ~ Phone*Cond*Finger, data = df, random=~1|PId)
summary(glht(lme_velocity, linfct=mcp(Cond = "Tukey")), test = adjusted(type = "bonferroni"))

Warning message in mcp2matrix(model, linfct = linfct):
“covariate interactions found -- default contrast might be inappropriate”



	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: lme.formula(fixed = Speed_2D ~ Phone * Cond * Finger, data = df, 
    random = ~1 | PId)

Linear Hypotheses:
                      Estimate Std. Error z value Pr(>|z|)    
walking - seated == 0   0.7587     0.1421    5.34 9.31e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Adjusted p values reported -- bonferroni method)


In [9]:
lme_velocity <- lme(Speed_2D ~ Phone*Cond*Finger, data = df, random=~1|PId)
summary(glht(lme_velocity, linfct=mcp(Phone = "Tukey")), test = adjusted(type = "bonferroni"))

Warning message in mcp2matrix(model, linfct = linfct):
“covariate interactions found -- default contrast might be inappropriate”



	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: lme.formula(fixed = Speed_2D ~ Phone * Cond * Finger, data = df, 
    random = ~1 | PId)

Linear Hypotheses:
                Estimate Std. Error z value Pr(>|z|)
OPO - N6 == 0 -6.977e-02  1.421e-01  -0.491    1.000
S3 - N6 == 0   6.306e-06  1.421e-01   0.000    1.000
S4 - N6 == 0  -2.089e-01  1.421e-01  -1.470    0.849
S3 - OPO == 0  6.978e-02  1.421e-01   0.491    1.000
S4 - OPO == 0 -1.391e-01  1.421e-01  -0.979    1.000
S4 - S3 == 0  -2.089e-01  1.421e-01  -1.470    0.849
(Adjusted p values reported -- bonferroni method)


In [10]:
lme_velocity <- lme(Speed_2D ~ Phone, data = df, random=~1|PId)
summary(glht(lme_velocity, linfct=mcp(Phone = "Tukey")), test = adjusted(type = "bonferroni"))


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: lme.formula(fixed = Speed_2D ~ Phone, data = df, random = ~1 | 
    PId)

Linear Hypotheses:
              Estimate Std. Error z value Pr(>|z|)    
OPO - N6 == 0 -0.28475    0.10569  -2.694  0.04233 *  
S3 - N6 == 0   0.60798    0.10569   5.753 5.27e-08 ***
S4 - N6 == 0  -0.36681    0.10569  -3.471  0.00312 ** 
S3 - OPO == 0  0.89273    0.10569   8.447  < 2e-16 ***
S4 - OPO == 0 -0.08206    0.10569  -0.776  1.00000    
S4 - S3 == 0  -0.97479    0.10569  -9.223  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Adjusted p values reported -- bonferroni method)


In [11]:
testInteractions(lme_velocity, pairwise=c("Phone"), adjustment="bonferroni")

,Value,Df,Chisq,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>
N6-OPO,0.28474991,1,7.2588769,4.233084e-02
N6-S3,-0.60797896,1,33.0917210,5.274735e-08
N6-S4,0.36680918,1,12.0454392,3.115155e-03
OPO-S3,-0.89272886,1,71.3479351,1.796827e-16
OPO-S4,0.08205927,1,0.6028338,1.000000e+00
S3-S4,0.97478814,1,85.0673148,1.730270e-19


# ANOVA only on walking 

In [12]:
df2 = df[df$Cond == "walking",]
aov2<-ezANOVA(data = df2,
       dv = Speed_2D,
       within = c(Finger, Phone),
       wid = PId,
       type = 3,
       detailed = F,
       return_aov = F)

aov2

$ANOVA
        Effect DFn DFd         F            p p<.05        ges
2       Finger   3  45 176.34856 6.913893e-25     * 0.46119759
3        Phone   3  45  38.06328 2.095655e-12     * 0.60449052
4 Finger:Phone   9 135  17.09139 1.424147e-18     * 0.08846915

$`Mauchly's Test for Sphericity`
        Effect            W            p p<.05
2       Finger 3.379927e-01 1.113513e-02     *
3        Phone 3.792840e-01 2.111302e-02     *
4 Finger:Phone 2.496683e-05 3.968012e-09     *

$`Sphericity Corrections`
        Effect       GGe        p[GG] p[GG]<.05       HFe        p[HF]
2       Finger 0.5979546 9.532536e-16         * 0.6739751 1.768508e-17
3        Phone 0.7109057 2.043270e-09         * 0.8321745 1.133358e-10
4 Finger:Phone 0.3038501 4.671368e-07         * 0.3782510 2.654709e-08
  p[HF]<.05
2         *
3         *
4         *

In [13]:
lme2 <- lme(Speed_2D ~ Phone, data = df2, random=~1|PId)
summary(glht(lme2, linfct=mcp(Phone = "Tukey")), test = adjusted(type = "bonferroni"))


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: lme.formula(fixed = Speed_2D ~ Phone, data = df2, random = ~1 | 
    PId)

Linear Hypotheses:
              Estimate Std. Error z value Pr(>|z|)    
OPO - N6 == 0 -0.50614    0.12729  -3.976 0.000420 ***
S3 - N6 == 0   1.07564    0.12729   8.450  < 2e-16 ***
S4 - N6 == 0  -0.54752    0.12729  -4.301 0.000102 ***
S3 - OPO == 0  1.58179    0.12729  12.426  < 2e-16 ***
S4 - OPO == 0 -0.04137    0.12729  -0.325 1.000000    
S4 - S3 == 0  -1.62316    0.12729 -12.751  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Adjusted p values reported -- bonferroni method)


# ANOVA only on sitting 

In [14]:
df2 = df[df$Cond == "seated",]
aov2<-ezANOVA(data = df2,
       dv = Speed_2D,
       within = c(Finger, Phone),
       wid = PId,
       type = 3,
       detailed = F,
       return_aov = F)
aov2

$ANOVA
        Effect DFn DFd         F            p p<.05        ges
2       Finger   3  45 54.666964 4.780691e-15     * 0.28212358
3        Phone   3  45  5.022307 4.352636e-03     * 0.12334280
4 Finger:Phone   9 135  4.135149 1.008635e-04     * 0.01556983

$`Mauchly's Test for Sphericity`
        Effect            W            p p<.05
2       Finger 0.2242952536 1.046689e-03     *
3        Phone 0.5906417827 2.057361e-01      
4 Finger:Phone 0.0001409085 2.125770e-06     *

$`Sphericity Corrections`
        Effect       GGe        p[GG] p[GG]<.05       HFe        p[HF]
2       Finger 0.5643185 2.125252e-09         * 0.6284213 3.112178e-10
3        Phone 0.7620949 9.553659e-03         * 0.9066469 5.917091e-03
4 Finger:Phone 0.3702641 8.636818e-03         * 0.4887030 3.657591e-03
  p[HF]<.05
2         *
3         *
4         *

In [15]:
lme2 <- lme(Speed_2D ~ Phone, data = df2, random=~1|PId)
summary(glht(lme2, linfct=mcp(Phone = "Tukey")), test = adjusted(type = "bonferroni"))


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: lme.formula(fixed = Speed_2D ~ Phone, data = df2, random = ~1 | 
    PId)

Linear Hypotheses:
              Estimate Std. Error z value Pr(>|z|)    
OPO - N6 == 0 -0.06336    0.05746  -1.103  1.00000    
S3 - N6 == 0   0.14032    0.05746   2.442  0.08760 .  
S4 - N6 == 0  -0.18610    0.05746  -3.239  0.00720 ** 
S3 - OPO == 0  0.20367    0.05746   3.545  0.00236 ** 
S4 - OPO == 0 -0.12274    0.05746  -2.136  0.19591    
S4 - S3 == 0  -0.32642    0.05746  -5.681 8.03e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Adjusted p values reported -- bonferroni method)
